# Multi-Factor Equity Portfolio

## Metheodology

Two different ways to build a multi-factor portfolio:

Bottom-up

Top-down

Different ways to construct a multi-factor portfolio:

Initial screen, multi-factor index

In [3]:
import os
import datetime
import yfinance as yf
from pandas.io.json import json_normalize
import requests
import pandas as pd
from pandas_datareader import data as pdr
import xlwings as xw

FMP_API_KEY = '67df1e8e1aeaf25ff767f4c00e34494d'


In [19]:
def pull_symbol_list():
    """
    All Companies ticker symbols available in Financial Modeling Prep.
    """
    url = f'https://financialmodelingprep.com/api/v3/stock/list?apikey={FMP_API_KEY}'
    symbol_list = requests.get(url)

    #falt and return result if reqeust success. return None otherwise
    if symbol_list.status_code == 200:
        flat_symbol_list = pd.json_normalize(symbol_list.json())
        # flat_symbol_list = flat_symbol_list[['symbol', 'name']]
        return flat_symbol_list

    return symbol_list.status_code
symbol_list = pull_symbol_list()
# symbol_list.to_csv('TICKER_LIST.csv', index=False)


In [37]:
us_symbols = symbol_list[(symbol_list['exchangeShortName'].isin(['NASDAQ','NYSE']) &symbol_list['type'].isin(['stock']))]

SYMBOL_INFO_ALL = pd.DataFrame()

for symbol in us_symbols.symbol.to_list():
    symbol_yf = yf.Ticker(symbol)
    symbol_info = pd.json_normalize(symbol_yf.info)
    SYMBOL_INFO_ALL = pd.concat([SYMBOL_INFO_ALL,symbol_info])
    
# SYMBOL_INFO_ALL.to_csv('SYMBOL_INFO_ALL.csv', index=False)

C:\Users\Harvey\AppData\Local\Temp/ipykernel_25640/2333623531.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  symbol_info = json_normalize(symbol_yf.info)


In [47]:
def pull_quarterly_financial_statement(ticker,statement_type):
    """
    All Companies ticker symbols available in Financial Modeling Prep.
    """
    if statement_type == 'IS':
        statement_type = 'income-statement'
    elif statement_type == 'BS':
        statement_type = 'balance-sheet-statement'
    elif statement_type == 'CF':
        statement_type = 'cash-flow-statement'
    
    url = f'https://financialmodelingprep.com/api/v3/{statement_type}/{ticker}?period=quarter&limit=400&apikey={FMP_API_KEY}'
    income_statement = requests.get(url)

    #falt and return result if reqeust success. return None otherwise
    if income_statement.status_code == 200:
        flat_income_statement = pd.json_normalize(income_statement.json())
        # flat_symbol_list = flat_symbol_list[['symbol', 'name']]
        return flat_income_statement

    return income_statement.status_code
# income_statement = pull_quarterly_financial_statement('AAPL', 'IS')
# symbol_list.to_csv('TICKER_LIST.csv', index=False)

In [64]:
def pull_daily_prices(ticker,start_date, end_date):
    """
    All Companies ticker symbols available in Financial Modeling Prep.
    """    
    url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={start_date}&to={end_date}&apikey={FMP_API_KEY}'
    daily_prices = requests.get(url)

    #falt and return result if reqeust success. return None otherwise
    if daily_prices.status_code == 200:
        flat_daily_prices = ''
        try:
            flat_daily_prices = pd.json_normalize(daily_prices.json(), 'historical', ['symbol'])
        except Exception as e:
            print(str(ticker)+str(e))
        # flat_symbol_list = flat_symbol_list[['symbol', 'name']]
        return flat_daily_prices

    return daily_prices.status_code
# daily_prices = pull_daily_prices('AAPL','2007-01-01','2022-01-24')
# symbol_list.to_csv('TICKER_LIST.csv', index=False)

In [48]:
SYMBOL_IS_ALL = pd.DataFrame()

for symbol in us_symbols.symbol.to_list():
    symbol_is = pull_quarterly_financial_statement(symbol, 'IS')
    SYMBOL_IS_ALL = pd.concat([SYMBOL_IS_ALL,symbol_is])

SYMBOL_IS_ALL.to_csv('SYMBOL_IS_ALL.csv', index=False)

In [54]:
SYMBOL_BS_ALL = pd.DataFrame()

for symbol in us_symbols.symbol.to_list():
    symbol_bs = pull_quarterly_financial_statement(symbol, 'BS')
    try:
        SYMBOL_BS_ALL = pd.concat([SYMBOL_BS_ALL,symbol_bs])
    except Exception as e:
        print(e)

SYMBOL_BS_ALL.to_csv('SYMBOL_BS_ALL.csv', index=False)

In [55]:
SYMBOL_CF_ALL = pd.DataFrame()

for symbol in us_symbols.symbol.to_list():
    symbol_cf = pull_quarterly_financial_statement(symbol, 'CF')
    try:
        SYMBOL_CF_ALL = pd.concat([SYMBOL_CF_ALL,symbol_cf])
    except Exception as e:
        print(e)

SYMBOL_CF_ALL.to_csv('SYMBOL_CF_ALL.csv', index=False)

In [65]:
SYMBOL_PRICE_ALL = pd.DataFrame()

for symbol in us_symbols.symbol.to_list():
    symbol_prices = pull_daily_prices(symbol,'2007-01-01','2022-01-24')
    try:
        SYMBOL_PRICE_ALL = pd.concat([SYMBOL_PRICE_ALL,symbol_prices])
    except Exception as e:
        print(e)

SYMBOL_PRICE_ALL.to_csv('SYMBOL_PRICE_ALL.csv', index=False)

AHC'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
CLIRW'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
DOTAU'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
ELECU'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
FNTEU'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
MICTW'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
MIIIU'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
MIIIW'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
MRDNW'historical'
cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid
ONSIZ'histor

In [4]:
HIST_PRICE = pd.read_csv('SYMBOL_PRICE_ALL.csv')
SYMBOL_CF_ALL = pd.read_csv('SYMBOL_CF_ALL.csv')
SYMBOL_BS_ALL = pd.read_csv('SYMBOL_BS_ALL.csv')
SYMBOL_IS_ALL = pd.read_csv('SYMBOL_IS_ALL.csv')
SYMBOL_INFO_ALL = pd.read_csv('SYMBOL_INFO_ALL.csv')

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (74,95,194,222) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [223]:
ticker='AAPL'
ticker_price = HIST_PRICE[HIST_PRICE.symbol==ticker]
ticker_is = SYMBOL_IS_ALL[SYMBOL_IS_ALL.symbol==ticker]
ticker_bs = SYMBOL_BS_ALL[SYMBOL_BS_ALL.symbol==ticker]
ticker_cf = SYMBOL_CF_ALL[SYMBOL_CF_ALL.symbol==ticker]
ticker_fundamental = pd.DataFrame()


In [224]:
ticker_is = ticker_is[['fillingDate', 'symbol', 'netIncome', 'weightedAverageShsOut', 'weightedAverageShsOutDil', 'operatingIncome', 'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense', 'interestExpense', 'eps', 'revenue', 'costOfRevenue']]
ticker_bs = ticker_bs[['fillingDate', 'symbol', 'shortTermDebt', 'longTermDebt', 'cashAndCashEquivalents', 'totalCurrentLiabilities', 'totalAssets', 'totalCurrentAssets']]
ticker_cf = ticker_cf[['fillingDate', 'symbol', 'capitalExpenditure', 'netCashProvidedByOperatingActivities', 'freeCashFlow']]

# ttm number
ticker_is.sort_values(by='fillingDate', inplace=True)
ticker_is_ttm = ticker_is.copy().groupby('symbol').rolling(4, on='fillingDate', min_periods=4)['netIncome','operatingIncome','depreciationAndAmortization', 'incomeTaxExpense', 'interestExpense', 'revenue', 'costOfRevenue'].sum()\
    .reset_index().rename(columns={'netIncome': 'netIncome_ttm', 'operatingIncome': 'operatingIncome_ttm', 'depreciationAndAmortization': 'depreciationAndAmortization_ttm',
                                   'incomeTaxExpense': 'incomeTaxExpense_ttm', 'interestExpense': 'interestExpense_ttm', 'revenue': 'revenue_ttm', 'costOfRevenue': 'costOfRevenue_ttm'})
ticker_is = ticker_is.merge(ticker_is_ttm, how='left', on=['fillingDate', 'symbol'])

ticker_cf.sort_values(by='fillingDate', inplace=True)
ticker_cf_ttm = ticker_cf.copy().groupby('symbol').rolling(4, on='fillingDate', min_periods=4)['netCashProvidedByOperatingActivities', 'capitalExpenditure'].sum()\
    .reset_index().rename(columns={'netCashProvidedByOperatingActivities': 'netCashProvidedByOperatingActivities_ttm', 'capitalExpenditure': 'capitalExpenditure_ttm'})
ticker_cf = ticker_cf.merge(ticker_cf_ttm, how='left', on=['fillingDate', 'symbol'])

# average total asset
ticker_bs.sort_values(by='fillingDate', inplace=True)
ticker_bs_avg = ticker_bs.copy().groupby('symbol').rolling(4, on='fillingDate', min_periods=4)['totalAssets', 'totalCurrentLiabilities', 'totalCurrentAssets'].mean()\
    .reset_index().rename(columns={'totalAssets': 'totalAssets_ttm_avg', 'totalCurrentLiabilities': 'totalCurrentLiabilities_ttm_avg', 'totalCurrentAssets': 'totalCurrentAssets_ttm_avg'})
ticker_bs = ticker_bs.merge(ticker_bs_avg, how='left', on=['fillingDate', 'symbol'])

# combine three statements
ticker_fundamental = ticker_is.merge(ticker_bs, how='inner', on=['fillingDate', 'symbol'])
ticker_fundamental = ticker_fundamental.merge(ticker_cf, how='inner', on=['fillingDate', 'symbol'])

ticker_overall = ticker_price.merge(ticker_fundamental, how='left', left_on=['date', 'symbol'], right_on=['fillingDate', 'symbol'])

# forward fill fundamental datacv
ticker_overall.sort_values(by='date', inplace=True)
ticker_overall.fillna(method='ffill', inplace=True)
ticker_overall.dropna(inplace=True)
# ticker_overall.to_csv('test.csv')

In [225]:
#   Value Factor
#   P/E

ticker_overall['pe_ratio'] = ticker_overall['close'] / ticker_overall['netIncome_ttm'] * ticker_overall['weightedAverageShsOutDil']

#   EBITDA/EV
#   EBITDA = NI + Int + Tax + D&A = operating income + D&A
ticker_overall['ebitda'] = ticker_overall['operatingIncome_ttm'] + ticker_overall['depreciationAndAmortization_ttm']

#   EV = Market Cap + Long-term debt + Short-term debt - Cash
ticker_overall['ev'] = ticker_overall['close'] * ticker_overall['weightedAverageShsOut'] + ticker_overall['longTermDebt'] + ticker_overall['shortTermDebt'] - ticker_overall['cashAndCashEquivalents']

ticker_overall['ebitda_ev_ratio'] = ticker_overall['ebitda'] / ticker_overall['ev']

#   FCF/P
#   FCF = CF Operating  - CAPEX (need to check)
# ticker_overall['tax_rate'] = ticker_overall['incomeTaxExpense']/ticker_overall['incomeBeforeTax']
# ticker_overall['tax_shield'] = ticker_overall['tax_rate'] * ticker_overall['interestExpense']
ticker_overall['fcf'] = ticker_overall['netCashProvidedByOperatingActivities_ttm'] + ticker_overall['capitalExpenditure_ttm']
ticker_overall['fcf_price_ratio'] = ticker_overall['fcf'] / ticker_overall['weightedAverageShsOutDil'] / ticker_overall['close']


## Quality Factor

FCF/Asset = FCF / Average Total Asset

ROA = NI_TTM / Average Total Asset

ROIC = EBIT / Invested Capital

Gross Margin = (Total Revenue TTM - COGS TTM) / Total Revenue TTM

Current Ratio = Average Current Asset / Average Current Liability

Inerest Coverage Ratio = 

In [226]:
#   Quality Factor

#   FCF/Total Assets
ticker_overall['fcf_asset_ratio'] = ticker_overall['fcf'] / ticker_overall['totalAssets_ttm_avg']

#   ROA
ticker_overall['roa'] = ticker_overall['netIncome_ttm'] / ticker_overall['totalAssets_ttm_avg']

#   ROIC = EBIT / Invested Capital
ticker_overall['ebit'] = ticker_overall['netIncome_ttm'] + ticker_overall['incomeTaxExpense_ttm'] + ticker_overall['interestExpense_ttm']
ticker_overall['invested_capital'] = ticker_overall['totalAssets_ttm_avg'] - ticker_overall['totalCurrentLiabilities_ttm_avg']

#   Gross Margin = (Total Revenue TTM - COGS TTM) / Total Revenue TTM
ticker_overall['gross_margin'] = ( ticker_overall['revenue_ttm'] - ticker_overall['costOfRevenue_ttm'] ) / ticker_overall['revenue_ttm']

#   Current Ratio
ticker_overall['current_ratio'] = ticker_overall['totalCurrentAssets_ttm_avg'] / ticker_overall['totalCurrentLiabilities_ttm_avg']

#   inerest coverage ratio
ticker_overall['interest_coverage'] = ticker_overall['ebit'] / ticker_overall['interestExpense_ttm']


In [227]:
# xw.books('testing.xlsx').sheets('Sheet3')
xw.books('testing.xlsx').sheets('Sheet3').range('A1').value = ticker_overall[ticker_overall['date']=='2021-10-29']

In [9]:
#   Trend Factor

#   Momentum Factor
SYMBOL_INFO_ALL = pd.read_csv('SYMBOL_INFO_ALL.csv')
SYMBOL_INFO_ALL[SYMBOL_INFO_ALL['industry'] == 'consumer staples ']

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (74,95,194,222) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,state,country,companyOfficers,website,...,equityHoldings.threeYearEarningsGrowthCat,equityHoldings.threeYearEarningsGrowth,equityHoldings.medianMarketCap,equityHoldings.priceToEarnings,equityHoldings.priceToBook,equityHoldings.priceToSalesCat,equityHoldings.priceToCashflowCat,address3,preMarketTime,preMarketChangePercent


In [10]:
SYMBOL_INFO_ALL.columns

Index(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city',
       'phone', 'state', 'country', 'companyOfficers', 'website',
       ...
       'equityHoldings.threeYearEarningsGrowthCat',
       'equityHoldings.threeYearEarningsGrowth',
       'equityHoldings.medianMarketCap', 'equityHoldings.priceToEarnings',
       'equityHoldings.priceToBook', 'equityHoldings.priceToSalesCat',
       'equityHoldings.priceToCashflowCat', 'address3', 'preMarketTime',
       'preMarketChangePercent'],
      dtype='object', length=225)